# Beyond Parallel Lines: A Practical Intro to Random Slope Models
### Unlock deeper insights with Random Slope Models and let the influence of your variables change across different groups, revealing more accurate and meaningful patterns in hierarchical data

# Introduction
#### In real-world data, the story is rarely that simple. A pattern that looks solid overall can behave very differently across regions, groups, or timeframes. And when you ignore that hidden variability? You risk making decisions based on incomplete - or worse, inaccurate - insights.
#### From campaign performance to clinical outcomes, context is everything. What works in one situation might fall flat in another. That's why Random Slope Models are game-changers. These models don't just show you a trend - they reveal how relationships shift across different parts of your data, giving you a clearer, more trustworthy picture.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

# --- Re-create the data with TRUE RANDOM SLOPES ---
np.random.seed(42) # For reproducibility

n_schools = 3
school_baselines = np.random.normal(loc=[60, 65, 70], scale=3, size=n_schools) # True random intercepts
# Let's add true random slope deviations for each school
true_school_slope_deviations = np.random.normal(loc=[-0.5, 0.2, 0.8], scale=0.3, size=n_schools)

students_per_school = 10
total_students = n_schools * students_per_school

student_ids = []
school_ids = []
app_hours = []
math_scores = []

fixed_slope_true = 2.0 # The overall average slope we are trying to estimate

for school_idx in range(n_schools):
    for student_idx in range(students_per_school):
        student_ids.append(f'S_{school_idx}_{student_idx}')
        school_id_str = f'School_{school_idx}' # Store as string for df
        school_ids.append(school_id_str)
        hours = np.random.uniform(1, 10)
        app_hours.append(hours)

        current_school_baseline = school_baselines[school_idx]
        current_school_slope_deviation = true_school_slope_deviations[school_idx]

        score = (
            current_school_baseline
            + (hours * (fixed_slope_true + current_school_slope_deviation))
            + np.random.normal(0, 5)
        )
        math_scores.append(score)

df = pd.DataFrame({
    'student_id': student_ids,
    'school_id': school_ids,
    'app_hours': app_hours,
    'math_score': math_scores
})

# --- Fit the Simple Linear Regression (OLS) Model (for comparison) ---
X_ols = df['app_hours']
y_ols = df['math_score']
X_ols_const = sm.add_constant(X_ols)
simple_model = sm.OLS(y_ols, X_ols_const)
simple_results = simple_model.fit()

# --- Fit the Mixed Linear Model (Random Intercept only, for comparison) ---
mixed_model_ri = smf.mixedlm("math_score ~ app_hours", df, groups=df["school_id"])
mixed_results_ri = mixed_model_ri.fit()

# --- Fit the Mixed Linear Model (Random Intercept AND Random Slope) ---
print("\n" + "="*80)
print("--- Fitting the Mixed Linear Model (Random Intercept AND Random Slope) ---")
print("="*80 + "\n")
mixed_model_rs = smf.mixedlm("math_score ~ app_hours", df, groups=df["school_id"], re_formula="~ app_hours")
mixed_results_rs = mixed_model_rs.fit()

print(mixed_results_rs.summary())

# --- Interpretation of Random Slope Model Output ---
print("\n" + "="*80)
print("--- Interpretation of Random Slope Model Output ---")
print("="*80 + "\n")

print("Let's look at the new details compared to the Random Intercept Only model:")

print("\n**1. Fixed Effects (Coefficients):**")
print(mixed_results_rs.params[['Intercept', 'app_hours']])
print("  - **`Intercept`**: This is the estimated **overall average baseline** math score when app_hours is zero. It's the $\\beta_0$ term.")
print("  - **`app_hours`**: This is the estimated **overall *average* effect** of one additional app hour on math score across all schools. It's the $\\beta_1$ term.")

print("\n**2. Random Effects (Covariance Matrix Components):**")
print("\nEstimated Random Effects Covariance Matrix (from cov_re attribute):")
print(mixed_results_rs.cov_re)

print("\nInterpretation of Random Effects components (from summary/cov_re):")
print("  - Look for rows/values related to `Group Var` (variance of random intercepts), `app_hours Var` (variance of random slopes), and `Intercept & app_hours Cov` (covariance between them).")
print("  - **`Group Var` (or Variance for Intercept):** This is the estimated variance of the random intercepts ($u_{0j}$). It tells you *how much group baselines vary* around the overall average intercept. A higher value suggests schools have significantly different starting points.")
print("  - **`app_hours Var` (or Variance for app_hours):** This is the estimated variance of the random slopes ($u_{1j}$). **This is the key new piece!** It tells you *how much the effect (slope) of app_hours varies* from school to school. A higher value means the app's impact (its slope) is very different across schools.")
print("  - **`Intercept & app_hours Cov` (or Covariance):** This is the estimated covariance between the random intercepts ($u_{0j}$) and random slopes ($u_{1j}$).")
print("    - A **positive covariance** suggests that schools with **higher baselines** (positive $u_{0j}$) tend to also have a **stronger app effect** (positive $u_{1j}$, steeper slope).")
print("    - A **negative covariance** suggests that schools with **higher baselines** tend to have a **weaker app effect** (negative $u_{1j}$, flatter slope).")
print("    - A covariance near zero suggests no strong relationship between a school's starting point and the app's effect within that school.")

print("\n**3. Scale (Residual Variance):**")
# print(f"  - **`Scale`**: {mixed_results_rs.scale:.3f} - This is the estimated variance of the individual errors (${{\\epsilon_{ij}}}$) after accounting for both random intercepts and random slopes. It represents the unexplained variation *within* students.")

# --- Visualization of Random Slope Model ---
print("\n" + "="*80)
print("--- Visualizing the Random Slope Model ---")
print("="*80 + "\n")

# Prepare data for plotting
x_range = np.linspace(df['app_hours'].min() - 0.5, df['app_hours'].max() + 0.5, 100)

# 1. OLS Predicted Line (for context)
ols_intercept = simple_results.params['const'].item()
ols_app_hours_coef = simple_results.params['app_hours'].item()
ols_predicted_line = ols_intercept + (ols_app_hours_coef * x_range)

# 2. MLM Random Slope Predicted Lines
# Get fixed effects (as scalars)
mlm_fixed_intercept_rs = mixed_results_rs.params['Intercept'].item()
mlm_fixed_app_hours_coef_rs = mixed_results_rs.params['app_hours'].item()


school_random_effects_rs = mixed_results_rs.random_effects

mlm_rs_lines_data = []
for school_id in df['school_id'].unique():
    # Access the Series of random effects for the current school_id from the dictionary
    school_specific_re = school_random_effects_rs[school_id]

    # --- FIX STARTS HERE ---
    u_0j = school_specific_re.iloc[0].item()
    u_1j = school_specific_re.iloc[1].item()
    # --- FIX ENDS HERE ---

    # Calculate the school-specific intercept and slope
    current_school_intercept = mlm_fixed_intercept_rs + u_0j
    current_school_slope = mlm_fixed_app_hours_coef_rs + u_1j

    # Predict scores using the school-specific intercept and slope
    predicted_scores = current_school_intercept + (current_school_slope * x_range)

    mlm_rs_lines_data.append(pd.DataFrame({
        'app_hours': x_range,
        'predicted_score': predicted_scores,
        'school_id': school_id
    }))
df_mlm_rs_lines = pd.concat(mlm_rs_lines_data)

# --- Plotting ---
plt.figure(figsize=(11, 8))

# Plot raw data points, colored by school
sns.scatterplot(data=df, x='app_hours', y='math_score', hue='school_id',
                palette='tab10', s=100, alpha=0.7, zorder=2)

# Plot OLS Regression Line (for comparison)
plt.plot(x_range, ols_predicted_line, color='red', linestyle='--',
         linewidth=2, label='OLS Overall Regression Line', zorder=1)

# Plot the overall fixed effects line (average intercept and average slope from MLM)
mlm_fixed_line_rs = mlm_fixed_intercept_rs + (mlm_fixed_app_hours_coef_rs * x_range)
plt.plot(x_range, mlm_fixed_line_rs, color='purple', linestyle=':',
         linewidth=2, label='MLM Fixed Effects Line (Overall Average)', zorder=1)

# Plot MLM Regression Lines for each school (these will NOT be parallel)
palette = sns.color_palette('tab10', n_schools)
school_colors = {f'School_{i}': palette[i] for i in range(n_schools)}

for school_id in df['school_id'].unique():
    subset = df_mlm_rs_lines[df_mlm_rs_lines['school_id'] == school_id]
    plt.plot(subset['app_hours'], subset['predicted_score'],
             color=school_colors[school_id], linestyle='-', linewidth=2,
             label=f'MLM School {school_id} Line (Adjusted)')

plt.title('Mixed Linear Model: Random Intercepts and Random Slopes')
plt.xlabel('App Hours')
plt.ylabel('Math Score')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(title='Legend', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("--- Visual Interpretation of Random Slope Plot ---")
print("="*80 + "\n")
print("1. **Data Points (Colored):** These show your student data, grouped by school. You'll likely notice the clusters now have different 'slants'.")
print("2. **OLS Overall Regression Line (Red Dashed):** This is still the 'one-size-fits-all' line, trying to fit all data points with a single slope and intercept.")
print("3. **MLM Fixed Effects Line (Purple Dotted):** This is the average line estimated by the Mixed Model. It represents the overall intercept and the overall average effect of app hours across all schools, *before* considering individual school deviations.")
print("4. **MLM School-Specific Regression Lines (Solid Lines, Colored by School):")
print("   - **Crucially, these lines are NOT parallel!** Each school has its own unique regression line, reflecting its own baseline (intercept) AND its own unique effect of app hours (slope).")
print("   - You'll likely see that some school lines are steeper (stronger app effect), while others are flatter (weaker app effect), and they are also shifted up or down based on their baseline.")
print("   - This directly visualizes the 'random slope' aspect – the relationship between app hours and math score is allowed to vary by school. The model has learned that the app's effectiveness isn't the same everywhere.")

print("\nThis type of model provides the most flexible and often most accurate representation of data where both the starting points and the effects of predictors vary across groups.")

# Conclusion
#### We've explored the strong features of Random Slope Models, going beyond simple regression and the limits of random intercept models. These advanced tools help us see how the effects of key variables change across different groups, giving a clearer and more detailed view of the data. This ability to uncover group-specific patterns is highly useful in many areas - from measuring educational outcomes to improving marketing strategies.
#### The goal is not just to find a connection, but to identify the most accurate one for each specific situation in your data. Random slope models help achieve this precision. By capturing the real-world complexity in your models, you uncover deeper and more meaningful insights.